In [2]:
# ['nigga', 'hate', 'love','ass','hell','better']
# #accuracy over all cross-validation folds: [0.6317343173431734, 0.618450184501845, 0.6140221402214022, 0.622140221402214, 0.6137370753323486]
# mean=0.62 std=0.01

# ['nigga', 'hate', 'love','ass','hell','better','bitch','fuck','dick']
# accuracy over all cross-validation folds: [0.6287822878228783, 0.6199261992619927, 0.6154981549815498, 0.6236162361623616, 0.6159527326440177]
# mean=0.62 std=0.00

# ['nigga', 'hate', 'love','ass','hell','better','bitch','fuck','dick','hey','shit','sexy','awesome']
# accuracy over all cross-validation folds: [0.6501845018450184, 0.6789667896678967, 0.6516605166051661, 0.6494464944649446, 0.6676514032496307]
# mean=0.66 std=0.01

# Having all individual words as features
# accuracy over all cross-validation folds: [0.7011070110701108, 0.6856088560885609, 0.6915129151291513, 0.7166051660516605, 0.7119645494830132]
# mean=0.70 std=0.01

#Same as above but remove all words appearing in less than 1% of all tweets, plus 2 and 3 grams
# accuracy over all cross-validation folds: [0.692250922509225, 0.6789667896678967, 0.6907749077490775, 0.7202952029520295, 0.7171344165435746]
# mean=0.70 std=0.02

#Same as above but remove all words appearing in less than 5% of all tweets, plus 2 and 3 grams
# accuracy over all cross-validation folds: [0.6464944649446495, 0.6686346863468635, 0.6575645756457564, 0.6553505535055351, 0.6506646971935007]
# mean=0.66 std=0.01

#Same as above but remove all words appearing in less than 0.5% of all tweets, plus 2 and 3 grams
# accuracy over all cross-validation folds: [0.6981549815498155, 0.6952029520295203, 0.6929889298892989, 0.7062730627306273, 0.7104874446085672]
# mean=0.70 std=0.01

#if I don`t clean the token which appears time too little, the word table of coefficients for nonhostile will be strange.(even have some japanese)
from collections import Counter
import glob
import hashlib
import io
import math
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import re
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import tarfile
import urllib.request
%matplotlib inline

In [3]:
def load_data(datafile):
    """
    Read your data into a single pandas dataframe where
    - each row is an instance to be classified
    (this could be a tweet, user, or news article, depending on your project)
    - there is a column called `label` which stores the class label (e.g., the true
      category for this row)
    """
    df = pd.read_csv(datafile)[['text', 'hostile']]
    df.columns = ['text', 'label']
    df['label'] = ['hostile' if i==1 else 'nonhostile' for i in df.label]
   # df['directed'] = ['directed' if i==1 else 'nondirected' for i in df.directed]
    return df

df = load_data('/Users/郑/elevate-osna-harassment/osna/data/data.csv')
df.head()

,text,label
0,@FlyGuyCree Nigga whatever one you gave me 🤦🏻‍♀️,nonhostile
1,@ArvindKejriwal . go to hell you ass hole.,hostile
2,@JohnJohnDaDon That “nigga” done lost his fuck...,hostile
3,@kane_tingle10 Can’t be fucked with them mate....,nonhostile
4,@JHarris_TheDon Its honestly better anyways. T...,nonhostile


In [4]:
df.label.value_counts()

hostile       3588
nonhostile    3186
Name: label, dtype: int64

In [5]:
# def make_features(df):
#     vec = DictVectorizer()
#     feature_dicts = []
#     words_to_track = ['nigga', 'hate', 'love','ass','hell','better','bitch','fuck','dick','hey','shit','sexy','awesome']
#     # will get different model for different features.
#     #words_to_track = ['you']
#     for i,row in df.iterrows():
#         features = {}
#         token_counts = Counter(re.sub('\W+', ' ', row['text'].lower()).split())
#         for w in words_to_track:
#             features[w] = token_counts[w]
#         feature_dicts.append(features)
#     X = vec.fit_transform(feature_dicts)
#     return X, vec
                
#X, vec = make_features(df)
min=[1,2,5,10]
vec = CountVectorizer()
X = vec.fit_transform(t for t in df['text'].values)

In [6]:
X

<6774x21018 sparse matrix of type '<class 'numpy.int64'>'
	with 99153 stored elements in Compressed Sparse Row format>

In [7]:
X.shape

(6774, 21018)

In [8]:
vec.vocabulary_

{'flyguycree': 6872,
 'nigga': 13062,
 'whatever': 20205,
 'one': 13543,
 'you': 20795,
 'gave': 7315,
 'me': 11925,
 'arvindkejriwal': 1599,
 'go': 7559,
 'to': 18883,
 'hell': 8195,
 'ass': 1648,
 'hole': 8394,
 'johnjohndadon': 9770,
 'that': 18423,
 'done': 5563,
 'lost': 11315,
 'his': 8331,
 'fucking': 7128,
 'mind': 12211,
 'smh': 17118,
 'kane_tingle10': 10088,
 'can': 3303,
 'be': 2165,
 'fucked': 7119,
 'with': 20371,
 'them': 18553,
 'mate': 11815,
 'wank': 20016,
 'over': 13708,
 'the': 18445,
 'royal': 15888,
 'family': 6520,
 'and': 1268,
 'too': 18964,
 'jharris_thedon': 9643,
 'its': 9276,
 'honestly': 8432,
 'better': 2357,
 'anyways': 1427,
 'these': 18639,
 'doctors': 5499,
 'doin': 5521,
 'crazy': 4469,
 'shit': 16688,
 'women': 20417,
 'who': 20246,
 'are': 1509,
 'delivering': 5106,
 'especially': 6229,
 'black': 2545,
 'statistics': 17609,
 'scare': 16248,
 'much': 12622,
 'lol': 11243,
 'austinmahone': 1783,
 'literally': 11114,
 'bitch': 2509,
 'forever': 6950,

In [9]:
X[0,5]

0

In [10]:
X[:,3].sum()

1

In [11]:
for word, idx in vec.vocabulary_.items():
    print('%20s\t%d' % (word, X[:,idx].sum()))

          flyguycree	1
               nigga	2340
            whatever	21
                 one	249
                 you	2622
                gave	17
                  me	612
      arvindkejriwal	1
                  go	222
                  to	1918
                hell	67
                 ass	674
                hole	15
       johnjohndadon	2
                that	1641
                done	50
                lost	30
                 his	321
             fucking	235
                mind	31
                 smh	35
       kane_tingle10	1
                 can	418
                  be	611
              fucked	29
                with	526
                them	204
                mate	24
                wank	3
                over	98
                 the	2716
               royal	1
              family	25
                 and	1691
                 too	190
      jharris_thedon	1
                 its	75
            honestly	28
              better	101
             anyways	7
               these	100

            garabage	1
               human	18
           something	90
            horrible	12
               conor	3
                fans	21
          patpokemon	1
           idamickel	1
                 nfl	2
             jaguars	1
                  qb	2
               makes	54
               grown	17
                 men	51
                kids	45
               rings	4
                dont	73
             anymore	30
                lmao	154
                 why	189
              acting	24
              earned	7
              mental	5
             illness	1
             ogcorey	1
          myndaqiw2w	1
         ryanjreilly	1
             imagine	22
          democratic	3
              nation	6
             america	25
             dignity	1
                show	52
           xian_bell	1
                 cuz	22
               named	9
                 joe	8
               hella	10
              bitter	8
               would	222
               sweat	3
           profusely	1
            

              fuckkk	1
           wizzofodd	1
            aocpress	1
        jackposobiec	6
                 bat	3
              parody	1
                acct	1
                seem	12
            rational	2
      kellyannepolls	4
     felicityhuffman	3
          cactus_dez	1
               south	2
              dakota	1
             current	5
               state	17
                  dx	1
        starvinlarry	1
      moogiemonsters	1
       lelenapeacock	1
       spicypurritos	1
     rosesdaughter61	1
     stillhopes4best	1
        keys_chronic	1
         chattyknana	1
           elianok10	1
      vickielbruton1	1
          rlamartini	1
        talkeetna101	1
         itmeuphoria	1
      mass_confuzion	1
       irishbrat1966	1
       painplayhouse	1
            kkittten	1
           justwinks	1
      heatherlinda11	1
            ladyag72	1
       brent_kellogg	1
     haveyoufoundou1	1
         angelsgal02	1
           pithypins	1
         ghostingeek	1
        jmkillingnyc	1
         

           instasnap	1
                 dey	7
              posted	9
         piersmorgan	10
               piers	1
             zzzzs01	1
       iamkneespears	1
            mentally	10
          challenged	2
                stan	4
       brokenspearit	1
                2019	9
                asib	1
             already	57
                sold	6
                  4m	1
              albums	2
                  ww	1
                week	27
           billboard	1
                 200	7
                hasn	7
                left	39
              bundle	2
            changing	6
         mvesilently	1
               upset	5
         information	7
        heterosexual	1
            messaged	2
            probably	44
               block	17
       citizenbomber	1
               uncle	16
                 tom	17
              cobley	1
       miiikeyyyyyyy	1
             married	13
                 sir	12
       woodythemover	1
                 yep	11
                days	27
                 ish	

                orgs	2
             funding	5
                fort	1
              winter	9
               great	48
            earldoe_	1
             _doranp	1
             cripple	35
     hannahbethanyxx	1
      livalexander08	1
             happily	2
             scrawny	1
             accepts	1
       cooke15justin	1
              offend	1
               inner	4
                flow	1
           naturally	6
                free	27
             squizxy	1
                flex	3
       jowilliams293	1
               least	28
                page	10
                kept	13
                muff	1
             covered	2
               hence	3
                news	8
              briefs	1
        just_a_fan46	1
        kimbermahree	1
       briennetarth6	1
                 yes	85
                fall	7
          inevitable	2
             worship	5
               alter	3
               alpha	3
       ofthornandmen	1
          nicadkins_	1
                lick	8
               balls	15
   

...

          connection	1
               grose	1
           davidrutz	1
        cherimoyacos	1
            croutean	1
              fellas	1
          itanimeirl	2
             classic	1
     crossbonezfreak	1
         jersey_jinx	1
            research	1
          pedophilia	2
            michaels	1
            patterns	1
                wade	2
               dodgy	1
        allanaharkin	1
              slogan	1
                 bid	1
         spynomobile	1
              hacker	1
          jlxhabeefj	1
              _aajed	1
            moso1022	1
       nightwing2303	1
     professionalism	1
         maddyxoxooo	1
             kavosyt	1
             petepax	1
            deadspin	2
              spends	1
             yoonkgs	1
         cancermdhvn	1
              ksksks	1
         supervising	1
      george65wilson	1
      tedermeatballs	1
          apologised	1
           lisamei62	1
             disrupt	1
          soverienty	1
                 sly	1
              snakey	1
         el

         nonsensical	1
              drivel	1
                loon	1
     jayzlyricsforev	1
             lineage	1
      negrostopheles	1
              bappin	1
              zooted	1
          onlyredmic	1
         phontigallo	1
               floss	1
          likebrandi	1
             airtime	1
          uppitykiki	1
        loveontheblu	1
           drewmixon	1
       _moniemontana	1
      dragonflyjonez	1
      everydaystrugg	1
          hzqojianpx	1
         cptnrwrpnts	1
         skysportspl	3
         ftbl_keirax	1
          vgvagabond	1
     crumycurmudgeon	1
         devilmaycry	1
           capcom_uk	1
         redgravekyo	1
     thestreetwriter	1
               tomqe	1
          gypsyotoko	1
           challenge	1
         travelinupe	1
          woofmeowww	1
             looneys	1
               shell	1
            journaay	1
       tooorangeyfor	1
            feminazi	1
         patronising	1
          addressing	1
         fixondennis	1
       caroline4real	1
          t

            jude5456	1
        windsorblade	1
               jjs74	1
               giver	1
         calebecarma	1
            illusion	1
             xlurkez	1
                 tay	1
     nonconsensually	1
         cafecomouro	1
              tenure	1
              raises	1
               maxed	1
     fefydislikesya1	1
           chibithot	1
             yogy255	1
         smltwnzenas	1
         blocboydeku	1
         zakmrcheeky	1
        reggieconway	1
                pree	1
          jecjt5icb7	1
          theravener	1
          ayr1lfhdew	1
      julius_starkey	1
            uzanator	1
              cvrdo_	1
         supacooldud	1
             slurped	1
            pr33omar	1
           christmas	1
               renee	1
         ivoriandoll	1
            finessed	1
                 115	1
             venessa	1
                 das	1
         obongroviel	1
           paul_fenz	1
           leftynero	1
      burningbrennan	1
     compoundamerica	1
      brianpmccarthy	1
      genob

             mumbles	1
             sweetly	1
            intimate	1
      mr_rogers_dick	1
           annamerch	1
       safehousemari	1
            samwhyte	1
               sadly	1
          bujubantzz	1
          jt84136344	1
             dlbro94	1
            dreamboy	1
       iam_peterposh	1
              gentle	1
           holysmoxe	1
      barbianastannn	1
        barbianastan	1
            feminism	1
         empowerment	1
                sksk	1
      nicholasadam20	1
            bubbache	1
            aidans67	1
      queenmachine67	1
     pmacgiollabhain	1
           daftlimmy	1
        kimsrichards	1
         ellamacavoy	1
          talentoniy	1
           yungchunn	1
               onika	1
               tanya	1
               maraj	1
          amorris010	1
        leebeard1984	1
          nonameishy	1
              height	1
     r3dnekteddybear	1
       heyitzchesney	1
          iangabelli	1
          spiffyjcyt	1
           turtorial	1
        unrepeteable	1
           

          hs3ekkwp59	1
           hitmankit	1
           strowger_	1
        sachawoodxxx	1
            strowger	1
       theneedledrop	1
        billmckibben	1
              slimey	1
               strip	1
           liberties	1
                 lma	1
     thisniggamondre	1
               frost	1
               april	1
                14th	1
        viewcamgirls	1
         ladyredwave	1
         maga45trump	1
           inclusive	1
          negotiable	1
         slaughtered	1
        christianity	1
          converting	1
         accompanied	1
             educate	1
              frizzy	1
          en8s01uifn	1
        thegorgieway	1
              spouts	1
            faceless	1
            nameless	1
             screams	1
      whoopiglowberg	1
           armypedia	1
          jonnyboehm	1
      starscream5269	1
         charlovato1	1
          gunschilli	1
             winding	1
           sensation	1
        ohmygoshgabs	1
         blakebrizzy	1
             hdctzar	1
       rebe

In [12]:
vec.get_feature_names()

['000',
 '0001',
 '00s',
 '011w3lrzq9',
 '02',
 '03',
 '0575',
 '06',
 '073',
 '08',
 '09',
 '0adswv0t7z',
 '0dkdxgfpwl',
 '0emvdxqddi',
 '0iw38bafg1',
 '0ltnp2wjlw',
 '0odxildrsq',
 '0qwcluhk4t',
 '0scu9pvweq',
 '0tto_spfx',
 '0vr1vdezoo',
 '10',
 '100',
 '1000',
 '1000000',
 '1000s',
 '1000v',
 '1000x',
 '101',
 '1017hammy',
 '10k',
 '10k_cash',
 '10ml',
 '10p',
 '10r',
 '11',
 '110',
 '115',
 '11am',
 '11stickyfingers',
 '12',
 '120',
 '125g',
 '12b',
 '12kjason',
 '13',
 '1337',
 '135',
 '137sol',
 '13ishis18',
 '13yo',
 '14',
 '148nft',
 '14am',
 '14freddyp',
 '14th',
 '14yr',
 '15',
 '150k',
 '157',
 '16',
 '160lb',
 '16troy1',
 '17',
 '17yrs',
 '18',
 '1800',
 '18_xxxbabes',
 '19',
 '190',
 '1916',
 '1943',
 '1950s',
 '1974',
 '1990s',
 '1995',
 '1awrencee',
 '1bigchek',
 '1blackgenius',
 '1chriskeating',
 '1djfirstclass',
 '1flyneen',
 '1ft',
 '1funnymike',
 '1future',
 '1fvq2z15sw',
 '1godbodyzay',
 '1godsgift_',
 '1godshad',
 '1goodtyme2013',
 '1grn_eyez',
 '1gunnagunna',
 '1

In [8]:
y = np.array(df.label)
Counter(y)

Counter({'nonhostile': 3186, 'hostile': 3588})

In [9]:
y[[0,5,12]]

array(['nonhostile', 'hostile', 'hostile'], dtype=object)

In [10]:
class_names = set(df.label)
for word, idx in vec.vocabulary_.items():
    for class_name in class_names:
        class_idx = np.where(y==class_name)[0]
        print('%20s\t%20s\t%d' % (word, class_name, X[class_idx, idx].sum()))

          flyguycree	          nonhostile	1
          flyguycree	             hostile	0
               nigga	          nonhostile	1418
               nigga	             hostile	922
            whatever	          nonhostile	9
            whatever	             hostile	12
                 one	          nonhostile	130
                 one	             hostile	119
                 you	          nonhostile	932
                 you	             hostile	1690
                gave	          nonhostile	9
                gave	             hostile	8
                  me	          nonhostile	322
                  me	             hostile	290
      arvindkejriwal	          nonhostile	0
      arvindkejriwal	             hostile	1
                  go	          nonhostile	85
                  go	             hostile	137
                  to	          nonhostile	888
                  to	             hostile	1030
                hell	          nonhostile	29
                hell	             hostile	38
   

             morning	          nonhostile	7
             morning	             hostile	5
             signoff	          nonhostile	0
             signoff	             hostile	1
                just	          nonhostile	243
                just	             hostile	246
                  so	          nonhostile	245
                  so	             hostile	229
                 ken	          nonhostile	1
                 ken	             hostile	2
                 let	          nonhostile	58
                 let	             hostile	59
            everyone	          nonhostile	26
            everyone	             hostile	35
                know	          nonhostile	148
                know	             hostile	144
                  he	          nonhostile	359
                  he	             hostile	503
                 get	          nonhostile	196
                 get	             hostile	177
                away	          nonhostile	18
                away	             hostile	25
      

               doesn	             hostile	46
                lose	          nonhostile	3
                lose	             hostile	7
                ball	          nonhostile	6
                ball	             hostile	10
               every	          nonhostile	36
               every	             hostile	46
                time	          nonhostile	101
                time	             hostile	97
                 fed	          nonhostile	1
                 fed	             hostile	3
                into	          nonhostile	28
                into	             hostile	43
                 him	          nonhostile	102
                 him	             hostile	162
            ijfine85	          nonhostile	1
            ijfine85	             hostile	0
       blessing_iyah	          nonhostile	1
       blessing_iyah	             hostile	0
                have	          nonhostile	215
                have	             hostile	253
                 had	          nonhostile	85
              

            peezeeb_	             hostile	1
                 man	          nonhostile	92
                 man	             hostile	87
              niggas	          nonhostile	29
              niggas	             hostile	43
                they	          nonhostile	243
                they	             hostile	191
                 ain	          nonhostile	77
                 ain	             hostile	67
               tryna	          nonhostile	16
               tryna	             hostile	8
               money	          nonhostile	24
               money	             hostile	32
                even	          nonhostile	95
                even	             hostile	107
                  50	          nonhostile	8
                  50	             hostile	6
               filed	          nonhostile	0
               filed	             hostile	1
          bankruptcy	          nonhostile	0
          bankruptcy	             hostile	1
             shortty	          nonhostile	0
             sho

               named	          nonhostile	2
               named	             hostile	7
                 joe	          nonhostile	4
                 joe	             hostile	4
               hella	          nonhostile	7
               hella	             hostile	3
              bitter	          nonhostile	3
              bitter	             hostile	5
               would	          nonhostile	99
               would	             hostile	123
               sweat	          nonhostile	2
               sweat	             hostile	1
           profusely	          nonhostile	1
           profusely	             hostile	0
               cause	          nonhostile	45
               cause	             hostile	39
              always	          nonhostile	52
              always	             hostile	47
                 hot	          nonhostile	22
                 hot	             hostile	12
            somebody	          nonhostile	8
            somebody	             hostile	6
                damn	  

...

         jdfromny206	          nonhostile	0
         jdfromny206	             hostile	1
            crissles	          nonhostile	0
            crissles	             hostile	1
       disheartening	          nonhostile	0
       disheartening	             hostile	1
           martherr2	          nonhostile	0
           martherr2	             hostile	1
      greatcountrytr	          nonhostile	0
      greatcountrytr	             hostile	1
      washingtonpost	          nonhostile	0
      washingtonpost	             hostile	1
          needybardi	          nonhostile	0
          needybardi	             hostile	1
         princequann	          nonhostile	0
         princequann	             hostile	1
           chartdata	          nonhostile	0
           chartdata	             hostile	1
       damianamathes	          nonhostile	1
       damianamathes	             hostile	0
        luckyluckyp_	          nonhostile	1
        luckyluckyp_	             hostile	0
        babeswezvese	          n

          r3lzwwn5zx	          nonhostile	1
          r3lzwwn5zx	             hostile	0
        chronicflood	          nonhostile	1
        chronicflood	             hostile	0
       stevenjonestc	          nonhostile	0
       stevenjonestc	             hostile	1
      thestarhalifax	          nonhostile	0
      thestarhalifax	             hostile	1
      heathermallick	          nonhostile	0
      heathermallick	             hostile	1
          lord_sugar	          nonhostile	1
          lord_sugar	             hostile	0
          daanstylez	          nonhostile	1
          daanstylez	             hostile	0
            atypical	          nonhostile	1
            atypical	             hostile	0
             _clamps	          nonhostile	0
             _clamps	             hostile	1
            viceland	          nonhostile	0
            viceland	             hostile	1
               desus	          nonhostile	0
               desus	             hostile	1
                mero	          n

           jamal2doe	             hostile	0
     michaelcmaronna	          nonhostile	0
     michaelcmaronna	             hostile	1
          unbearable	          nonhostile	0
          unbearable	             hostile	1
           apologist	          nonhostile	0
           apologist	             hostile	1
  thereckoningisnigh	          nonhostile	0
  thereckoningisnigh	             hostile	1
      seana_sheridan	          nonhostile	0
      seana_sheridan	             hostile	1
      norinco_milled	          nonhostile	1
      norinco_milled	             hostile	0
                arab	          nonhostile	1
                arab	             hostile	0
             manlets	          nonhostile	1
             manlets	             hostile	0
            lanklets	          nonhostile	1
            lanklets	             hostile	0
           inbetween	          nonhostile	1
           inbetween	             hostile	0
               lanky	          nonhostile	1
               lanky	           

      faithfullyx0x0	          nonhostile	0
      faithfullyx0x0	             hostile	1
       followthelila	          nonhostile	0
       followthelila	             hostile	1
          okay_themb	          nonhostile	0
          okay_themb	             hostile	1
         gebhoyfitzy	          nonhostile	1
         gebhoyfitzy	             hostile	0
            baz_2511	          nonhostile	1
            baz_2511	             hostile	0
            parkhead	          nonhostile	1
            parkhead	             hostile	0
           thejay305	          nonhostile	0
           thejay305	             hostile	1
         easportshoe	          nonhostile	1
         easportshoe	             hostile	0
        zikohercules	          nonhostile	1
        zikohercules	             hostile	0
            hercules	          nonhostile	1
            hercules	             hostile	0
       danielcblight	          nonhostile	0
       danielcblight	             hostile	1
        hudson_derek	          n

         pacojonesfl	             hostile	1
         bitcoinbuil	          nonhostile	1
         bitcoinbuil	             hostile	0
               towel	          nonhostile	1
               towel	             hostile	0
           clubdoesn	          nonhostile	1
           clubdoesn	             hostile	0
             shinola	          nonhostile	1
             shinola	             hostile	0
           lwthesun_	          nonhostile	0
           lwthesun_	             hostile	1
          jesyfanaus	          nonhostile	0
          jesyfanaus	             hostile	1
      glamnationlive	          nonhostile	0
      glamnationlive	             hostile	1
        salutekweens	          nonhostile	0
        salutekweens	             hostile	1
              finnmp	          nonhostile	0
              finnmp	             hostile	1
       moscowhitetsb	          nonhostile	0
       moscowhitetsb	             hostile	1
         goalkeeping	          nonhostile	0
         goalkeeping	           

In [11]:
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier

hidden_layer_sizes = [(10, ), (50, ), (100, ), (200, )]
alpha = [.001, .0001, .00001]
clf01 = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes[0], alpha=0.0001)
clf02 = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes[1], alpha=0.0001)
clf03 = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes[2], alpha=0.0001)
clf04 = MLPClassifier(hidden_layer_sizes=hidden_layer_sizes[3], alpha=0.0001)
clf05 = MLPClassifier(hidden_layer_sizes=(100, ), alpha=alpha[0])
clf06 = MLPClassifier(hidden_layer_sizes=(100, ), alpha=alpha[1])
clf07 = MLPClassifier(hidden_layer_sizes=(100, ), alpha=alpha[2])

min_samples_leaf = [1, 3, 5]
n_estimators = [100, 200, 300]
clf11 = RandomForestClassifier(min_samples_leaf=min_samples_leaf[0],n_estimators=100)
clf12 = RandomForestClassifier(min_samples_leaf=min_samples_leaf[1],n_estimators=100)
clf13 = RandomForestClassifier(min_samples_leaf=min_samples_leaf[2],n_estimators=100)
clf14 = RandomForestClassifier(min_samples_leaf=1,n_estimators=n_estimators[0])
clf15 = RandomForestClassifier(min_samples_leaf=1,n_estimators=n_estimators[1])
clf16 = RandomForestClassifier(min_samples_leaf=1,n_estimators=n_estimators[2])
#clf = LogisticRegression(multi_class='auto',C=1, penalty='l2')

clf=[clf01,clf02,clf03,clf04,clf05,clf06,clf07,clf11,clf12,clf13,clf14,clf15,clf16]


In [ ]:
clf.fit(X, y)

In [167]:
clf.coef_

array([[-0.27731524,  0.17256148,  0.19467686, ...,  0.42091954,
         0.10450873, -0.10310452]])

In [62]:
coef = [-clf.coef_[0], clf.coef_[0]]
print(coef)

[array([ 0.31187423, -0.08525184, -0.07385308, ...,  0.0616842 ,
        0.0616842 ,  0.0616842 ]), array([-0.31187423,  0.08525184,  0.07385308, ..., -0.0616842 ,
       -0.0616842 , -0.0616842 ])]


In [63]:
clf.classes_

array(['hostile', 'nonhostile'], dtype=object)

In [155]:
for ci, class_name in enumerate(clf.classes_):
    print('coefficients for %s' % class_name)
    display(pd.DataFrame([coef[ci]], columns=vec.get_feature_names()))

coefficients for hostile


,about,actually,after,again,ain,all,all the,already,also,always,...,you,you are,you can,you don,you have,you know,you re,your,your ass,yourself
0,0.142367,-0.157571,-0.255209,0.203245,0.007781,0.093158,-0.201695,-0.637633,0.492342,-0.147612,...,0.247989,0.445111,-0.43247,0.126951,-0.615671,-0.405049,-0.279251,0.489423,-0.104745,-0.244697


coefficients for nonhostile


,about,actually,after,again,ain,all,all the,already,also,always,...,you,you are,you can,you don,you have,you know,you re,your,your ass,yourself
0,-0.142367,0.157571,0.255209,-0.203245,-0.007781,-0.093158,0.201695,0.637633,-0.492342,0.147612,...,-0.247989,-0.445111,0.43247,-0.126951,0.615671,0.405049,0.279251,-0.489423,0.104745,0.244697


In [156]:
features = vec.get_feature_names()
for ci, class_name in enumerate(clf.classes_):
    print('top features for class %s' % class_name)
    for fi in coef[ci].argsort()[::-1][:10]: # descending order.
        print('%20s\t%.2f' % (features[fi], coef[ci][fi]))

top features for class hostile
                twat	2.80
                cunt	2.60
     realdonaldtrump	2.39
              retard	2.36
                shut	2.02
              faggot	1.85
            fuck off	1.78
               pussy	1.73
            fuck you	1.28
                 cum	1.24
top features for class nonhostile
             need to	1.28
                holy	1.14
                 mad	1.03
             calling	1.01
            what the	1.00
                love	0.95
            the only	0.85
            going to	0.81
                same	0.77
               video	0.75


In [217]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []
for train, test in kf.split(X):
    clf.fit(X[train], y[train])
    pred = clf.predict(X[test])
    accuracies.append(accuracy_score(y[test], pred))
    
print('accuracy over all cross-validation folds: %s' % str(accuracies))
print('mean=%.2f std=%.2f' % (np.mean(accuracies), np.std(accuracies)))

accuracy over all cross-validation folds: [0.7121771217712177, 0.7114391143911439, 0.7129151291512915, 0.7269372693726938, 0.7171344165435746]
mean=0.72 std=0.01


In [195]:
d1 = ({
    'min_df': [1, 2, 5, 10],
    'Accuracy': [0.72, 0.72, 0.71, 0.71],
})
d2 = ({
    'max_df': [1, .95, .8],
    'Accuracy': [0.72, 0.71, 0.71],
})
d3 = ({
    'ngram_range': [(1,1), (1,2), (1,3)],
    'Accuracy': [0.72, 0.72, 0.72],
})
d4 = ({
    'c': [.1, 1, 5, 10],
    'Accuracy': [0.70, 0.72, 0.70, 0.70],
})
d5 = ({
    'penalty': ['l1','l2'],
    'Accuracy': [0.72, 0.72],
})
df1 = pd.DataFrame(d1)
df2 = pd.DataFrame(d2)
df3 = pd.DataFrame(d3)
df4 = pd.DataFrame(d4)
df5 = pd.DataFrame(d5)
print(df1)
print(df2)
print(df3)
print(df4)
print(df5)

   min_df  Accuracy
0       1      0.72
1       2      0.72
2       5      0.71
3      10      0.71
   max_df  Accuracy
0    1.00      0.72
1    0.95      0.71
2    0.80      0.71
  ngram_range  Accuracy
0      (1, 1)      0.72
1      (1, 2)      0.72
2      (1, 3)      0.72
      c  Accuracy
0   0.1      0.70
1   1.0      0.72
2   5.0      0.70
3  10.0      0.70
  penalty  Accuracy
0      l1      0.72
1      l2      0.72


In [12]:
mean = []
std = []
arr_accuracies = []

for clf in clf:
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = []
    for train, test in kf.split(X):
        clf.fit(X[train], y[train])
        pred = clf.predict(X[test])
        accuracies.append(accuracy_score(y[test], pred))
    arr_accuracies.append(str(accuracies))
    mean.append(np.mean(accuracies))
    std.append(np.std(accuracies))
    
print('%5s'%'hidden_layer_sizes', '%12s'%'accuracy')
for i in range(0,4):
    print('%5s%10.2f'%(hidden_layer_sizes[i], mean[i]))
    
print('alpha', '%10s'%'accuracy')
for i in range(0,3):
    print('%5s%10.2f'%(alpha[i], mean[i]))

print('min_samples_leaf', '%10s'%'accuracy')
for i in range(0,3):
print('%5s%10.2f'%(min_samples_leaf[i], mean[i]))
    
print('n_estimators', '%10s'%'accuracy')
for i in range(0,3):
    print('%5s%10.2f'%(n_estimators[i], mean[i]))

hidden_layer_sizes     accuracy
(10,)      0.69
(50,)      0.69
(100,)      0.69
(200,)      0.69
alpha   accuracy
0.001      0.69
0.0001      0.69
1e-05      0.69
min_samples_leaf   accuracy
    1      0.69
    3      0.69
    5      0.69
n_estimators   accuracy
  100      0.69
  200      0.69
  300      0.69
